In [20]:
import pandas as pd
import requests
import os
import gmaps
import numpy as np
from config import gkey

In [21]:
#Import the csv from part one
weather_city_csv = os.path.join('..','WeatherPy','output','city_weather_data.csv')
weather_df = pd.read_csv(weather_city_csv)
weather_df.head()

,City,Country,Latitude,Longitude,Temperature (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Datetime
0,Paradise Valley,US,33.5312,-111.9427,24.39,89,90,0.89,1627217309
1,Ribeira Grande,PT,38.5167,-28.7000,23.22,78,40,3.09,1627217200
2,Hambantota,LK,6.1241,81.1185,30.19,74,75,7.72,1627217310
3,Ushuaia,AR,-54.8000,-68.3000,3.81,81,40,4.63,1627217260
4,Chimoio,MZ,-19.1164,33.4833,15.92,67,75,5.66,1627217312


In [22]:
 #Configure gmaps and api call info
 gmaps.configure(api_key=gkey)
 figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [23]:
#Create Humidity heatmap
humidity = weather_df['Humidity (%)'].astype(float)
locations = weather_df[['Latitude','Longitude']].astype(float)
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=[0,0])
#Find the maximum humidity available
max_humidity = weather_df['Humidity (%)'].max().astype(float)
# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,point_radius=4,dissipating=False,max_intensity=max_humidity)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [24]:
#Narrow Down dataframe based on:
#Temperature between 22 and 24C
#Wind Speed less than 8 MPH
#Cloudiness of 0%
#Make it a copy so we can add the hotel column later on
ideal_df = weather_df.loc[(weather_df['Temperature (C)'] > 22) & (weather_df['Temperature (C)'] < 24) & (weather_df['Wind Speed (m/s)'] < 8) & (weather_df['Cloudiness (%)'] == 0)].copy().reset_index(drop=True)
print(len(ideal_df))
ideal_df = ideal_df[['City','Country','Latitude','Longitude']]
ideal_df

8


,City,Country,Latitude,Longitude
0,Cape Town,ZA,-33.9258,18.4232
1,Kasempa,ZM,-13.4584,25.8338
2,Port Elizabeth,ZA,-33.9180,25.5701
3,São Francisco,BR,-15.9486,-44.8644
4,São João da Barra,BR,-21.6403,-41.0511
5,Henties Bay,NaN,-22.1160,14.2845
6,Pringsewu,ID,-5.3582,104.9744
7,Marquette,US,46.5333,-87.6335


In [30]:
params = {
    "radius":5000,
    "types":"lodging",
    "key":gkey
}
num_possible_locations = len(ideal_df)
for index,row in ideal_df.iterrows():
    lat=row['Latitude']
    long = row['Longitude']
    city=row['City']
    params['location'] = f"{lat},{long}"
    try:
        response = requests.get(base_url,params=params).json()
        results = response['results']
        ideal_df.loc[index,'Hotel Name'] = results[0]['name']
        print(f"Found Hotel info for {city} | {index + 1} of {num_possible_locations}")
    except:
        print(f"Unable to find hotel information in {city} | {index + 1} of {num_possible_locations}")

Found Hotel info for Cape Town | 1 of 8
Found Hotel info for Kasempa | 2 of 8
Found Hotel info for Port Elizabeth | 3 of 8
Found Hotel info for São Francisco | 4 of 8
Found Hotel info for São João da Barra | 5 of 8
Found Hotel info for Henties Bay | 6 of 8
Found Hotel info for Pringsewu | 7 of 8
Found Hotel info for Marquette | 8 of 8


In [31]:
#Drop any NA values where we couldn'`t find hotel information
ideal_df.dropna()

,City,Country,Latitude,Longitude,Hotel Name
0,Cape Town,ZA,-33.9258,18.4232,Southern Sun Waterfront Cape Town
1,Kasempa,ZM,-13.4584,25.8338,Lolelunga Forest Reserve
2,Port Elizabeth,ZA,-33.9180,25.5701,39 On Nile Guest House
3,São Francisco,BR,-15.9486,-44.8644,Hotel Supremo
4,São João da Barra,BR,-21.6403,-41.0511,Pousada Porto De Canoas
6,Pringsewu,ID,-5.3582,104.9744,Urban Style by Front One
7,Marquette,US,46.5333,-87.6335,Al Quaal Lodge


In [32]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]


In [33]:
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…